In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
voice = pd.read_csv('../Resources/voice.csv')
voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [4]:
#X = data , y = target
X = voice.drop('label',axis=1)
y = voice['label']
print(X.shape,y.shape)

(3168, 20) (3168,)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# step1 : Labelencode data set 
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# step 2: conver encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
# create deep learning model 
from keras.models import Sequential
from keras.layers import Dense

# reate model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=2,activation='softmax'))

In [12]:
# complie and fit the model 
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(
X_train_scaled, y_train_categorical, epochs=60,
shuffle=True, verbose=2)

Epoch 1/60
 - 0s - loss: 0.2752 - acc: 0.9019
Epoch 2/60
 - 0s - loss: 0.0865 - acc: 0.9747
Epoch 3/60
 - 0s - loss: 0.0761 - acc: 0.9756
Epoch 4/60
 - 0s - loss: 0.0635 - acc: 0.9781
Epoch 5/60
 - 0s - loss: 0.0617 - acc: 0.9811
Epoch 6/60
 - 0s - loss: 0.0550 - acc: 0.9802
Epoch 7/60
 - 0s - loss: 0.0500 - acc: 0.9840
Epoch 8/60
 - 0s - loss: 0.0488 - acc: 0.9827
Epoch 9/60
 - 0s - loss: 0.0467 - acc: 0.9836
Epoch 10/60
 - 0s - loss: 0.0432 - acc: 0.9840
Epoch 11/60
 - 0s - loss: 0.0402 - acc: 0.9844
Epoch 12/60
 - 0s - loss: 0.0379 - acc: 0.9844
Epoch 13/60
 - 0s - loss: 0.0346 - acc: 0.9870
Epoch 14/60
 - 0s - loss: 0.0336 - acc: 0.9865
Epoch 15/60
 - 0s - loss: 0.0304 - acc: 0.9878
Epoch 16/60
 - 0s - loss: 0.0274 - acc: 0.9899
Epoch 17/60
 - 0s - loss: 0.0256 - acc: 0.9916
Epoch 18/60
 - 0s - loss: 0.0259 - acc: 0.9903
Epoch 19/60
 - 0s - loss: 0.0257 - acc: 0.9903
Epoch 20/60
 - 0s - loss: 0.0231 - acc: 0.9937
Epoch 21/60
 - 0s - loss: 0.0199 - acc: 0.9945
Epoch 22/60
 - 0s - lo

In [14]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(
f'Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}')

Normal Neural Network - Loss: 0.059738493535662955, Accuracy: 0.9873737373737373


In [18]:
# Make prediction
encoded_predictions = model.predict_classes(X_test_scaled[:5])
print(encoded_predictions)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

[0 0 1 0 1]


/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
print(f'Predicted classes: {prediction_labels}')
print(f'actual Labels: {list(y_test[:5])}')

Predicted classes: ['female' 'female' 'male' 'female' 'male']
actual Labels: ['female', 'female', 'male', 'female', 'male']
